In [ ]:
import pandas as pd
import os
import gc
from tqdm import tqdm

In [ ]:
ROOT_DIR = "../input/mlb-player-digital-engagement-forecasting"

## COMPUTING PLAYER MEAN

In [ ]:
tgt_df = pd.read_csv("../input/mlb-data/target.csv")

In [ ]:
tgt_df.head()

In [ ]:
player_mean = tgt_df.groupby("playerId")[["target1","target2","target3","target4"]].median().reset_index()
gc.collect()
print(player_mean.shape)

In [ ]:
player_mean.head()

In [ ]:
TGTCOLS = ["target1","target2","target3","target4"]
imp = tgt_df[TGTCOLS].median()

In [ ]:
def process_pred(df):
    df = df.reset_index()
    df["playerId"] = df["date_playerId"].apply(lambda x: int( x.split("_")[1] ) )
    df.drop(["target1","target2","target3","target4"], axis=1, inplace=True)
    df = df.merge(player_mean, on="playerId", how="left")
    df.drop(["playerId","date"], axis=1, inplace=True)
    #df = df.fillna(0.)
    for col in TGTCOLS:
        df[col] = df[col].fillna(imp[col])
    return df
#===================

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df = process_pred(sample_prediction_df)
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df.head()

In [ ]:
sample_prediction_df["target1"] = sample_prediction_df["target1"].fillna(imp["target1"])